In [6]:
import re

path = "<src>"
with open(path, encoding='utf-8') as f:
    ls = f.readlines()

In [7]:
mc1 = "C stack start"
mc2 = "C stack end"
mp1 = "PHP stack start"
mp2 = "PHP stack end"

start = 0
got_c_start = 1
got_c_end = 2
got_php_start = 3

stacks_complete = []
stacks_incomplete = []

buff_c = []
buff_php = []
buff_others = []
state = start
for l in ls:
    if mc1 in l:
        if state != start:
            stacks_incomplete.append((state, buff_c, buff_php, buff_others, l))
        state = got_c_start
        buff_c = []
        buff_php = []
        buff_others = []
    elif mc2 in l:
        if state != got_c_start:
            stacks_incomplete.append((state, buff_c, buff_php, buff_others, l))
            state = start
            buff_c = []
            buff_php = []
            buff_others = []
        else:
            state = got_c_end
    elif mp1 in l:
        if state != got_c_end:
            stacks_incomplete.append((state, buff_c, buff_php, buff_others, l))
            state = start
            buff_c = []
            buff_php = []
            buff_others = []
        else:
            state = got_php_start
    elif mp2 in l:
        if state != got_php_start:
            stacks_incomplete.append((state, buff_c, buff_php, buff_others, l))
            state = start
        else:
            stacks_complete.append((buff_c, buff_php, buff_others))
            state = start
        buff_c = []
        buff_php = []
        buff_others = []
    else:
        if state >= got_php_start and l.startswith("$"):
            buff_php.append(l)
        elif state >= got_c_start and l.startswith("#"):
            buff_c.append(l)
        else:
            buff_others.append(l)

In [8]:
stacks_complete_v2 = []
for c1, php1, _ in stacks_complete:
    c2 = []
    for l in c1:
        s1 = re.split("^#\d+\s+", l, maxsplit=2)[1]
        s2_a = re.split("^0x[a-f\d]+\s+in\s+", s1, maxsplit=2)
        s2 = s2_a[len(s2_a) - 1]
        s3 = s2.split(" (", maxsplit=1)
        if len(s3) != 2:
            print(s1, s2_a, s3, l)
            continue
        if s3[0] == '??':
            continue
        c2.append(s3[0])
    php2 = []
    for l in php1:
        s1 = l.split(' = ', maxsplit=1)[1]
        if not s1.startswith("0x"):
            continue
        s2 = re.split("0x[a-f\d]+ ", s1)[1]
        s3 = s2.removeprefix('"').removesuffix('"\n')
        php2.append(s3)
    stacks_complete_v2.append((c2, php2))

In [9]:
def f(d, v):
    v_s = ';'.join(v)
    d[v_s] = d.setdefault(v_s, 0) + 1


d = {}
for c, php0 in stacks_complete_v2:
    php = [x + '::' for x in php0]
    last = -1
    for i, l in enumerate(reversed(c)):
        if 'execute_ex' in l:
            last = i
    if last == -1:
        v = c + php
    else:
        last = len(c) - last - 1
        v = c[:last] + php + c[last:]
    f(d, list(reversed(v)))

with open('./stacks.c.php', mode='w') as f:
    for k, v in d.items():
        f.write('%s %d\n' % (k, v))

In [10]:
def f(d, v):
    v_s = ';'.join(v)
    d[v_s] = d.setdefault(v_s, 0) + 1


d = {}
for c, php0 in stacks_complete_v2:
    php = [x + '::' for x in php0]
    last = -1
    for i, l in enumerate(reversed(c)):
        if 'execute_ex' in l:
            last = i
    if last == -1:
        v = c + php
    else:
        last = len(c) - last - 1
        v = c[:last] + php + c[last:]
    f(d, list(reversed(v)))

with open(f"{path}.stacks", mode='w') as f:
    for k, v in d.items():
        f.write('%s %d\n' % (k, v))